# Evaluation with RAGAS and Advanced Retrieval Methods Using LangChain

In the following notebook we'll discuss a major component of LLM Ops:

- Evaluation

We're going to be leveraging the [RAGAS]() framework for our evaluations today as it's becoming a standard method of evaluating (at least directionally) RAG systems.

We're also going to discuss a few more powerful Retrieval Systems that can potentially improve the quality of our generations!

Let's start as we always do: Grabbing our dependencies!

In [1]:
# !pip install -U -q langchain openai ragas arxiv pymupdf chromadb wandb tiktoken

In [2]:
import os
import openai
from langchain_community.document_loaders import PyPDFLoader
from werkzeug.utils import secure_filename
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.schema import Document
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter



es_cloud_id="id"
es_api_key="key"
elastic_search_client=Elasticsearch(cloud_id=es_cloud_id, api_key=es_api_key, timeout=300)
os.environ["OPENAI_API_KEY"] = 'key'

/home/asura/miniconda3/envs/qdoc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_42910/4238627262.py:23: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  elastic_search_client=Elasticsearch(cloud_id=es_cloud_id, api_key=es_api_key, timeout=300)


In [3]:
#loader
loader = PyPDFLoader("diary.pdf")
docs = loader.load()
len(docs)



/home/asura/miniconda3/envs/qdoc/lib/python3.12/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


21

In [4]:
for doc in docs:
  print(doc.metadata)

{'source': 'diary.pdf', 'page': 0}
{'source': 'diary.pdf', 'page': 1}
{'source': 'diary.pdf', 'page': 2}
{'source': 'diary.pdf', 'page': 3}
{'source': 'diary.pdf', 'page': 4}
{'source': 'diary.pdf', 'page': 5}
{'source': 'diary.pdf', 'page': 6}
{'source': 'diary.pdf', 'page': 7}
{'source': 'diary.pdf', 'page': 8}
{'source': 'diary.pdf', 'page': 9}
{'source': 'diary.pdf', 'page': 10}
{'source': 'diary.pdf', 'page': 11}
{'source': 'diary.pdf', 'page': 12}
{'source': 'diary.pdf', 'page': 13}
{'source': 'diary.pdf', 'page': 14}
{'source': 'diary.pdf', 'page': 15}
{'source': 'diary.pdf', 'page': 16}
{'source': 'diary.pdf', 'page': 17}
{'source': 'diary.pdf', 'page': 18}
{'source': 'diary.pdf', 'page': 19}
{'source': 'diary.pdf', 'page': 20}


### Creating an Index

Let's use a naive index creation strategy of just using `RecursiveCharacterTextSplitter` on our documents and embedding each into our `VectorStore` using `OpenAIEmbeddings()`.

- [`RecursiveCharacterTextSplitter()`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html)
- [`Chroma`](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.chroma.Chroma.html?highlight=chroma#langchain.vectorstores.chroma.Chroma)
- [`OpenAIEmbeddings()`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html?highlight=openaiembeddings#langchain-embeddings-openai-openaiembeddings)

In [5]:


def get_hierarchical_chunks(pages):
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    
    final_chunks = []
    for page in pages:
        md_header_splits = markdown_splitter.split_text(page.page_content)
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            chunk_overlap=400,
            length_function=len,
            separators=["\n\n", "\n", " ", ""]
        )

        for doc in md_header_splits:
            smaller_chunks = text_splitter.split_text(doc.page_content)
            for chunk in smaller_chunks:
                final_chunks.append(Document(
                    page_content=chunk,
                    metadata={
                        "source": page.metadata.get("source", ""),
                        "page": page.metadata.get("page", ""),
                        "header": " > ".join([doc.metadata.get(f"Header {i}", "") for i in range(1, 4) if f"Header {i}" in doc.metadata])
                    }
                ))

    return final_chunks

def get_text_chunks(pages, user_session):
    # Assuming `pages` is a list of Document objects, each representing a page of the document
    all_chunks = []

    # Iterate over each page and apply hierarchical chunking
    full_text = ""
    for page in pages:
        # Get hierarchical chunks
        hierarchical_chunks = get_hierarchical_chunks([page])
        all_chunks.extend(hierarchical_chunks)
        full_text += page.page_content
    
    if not os.path.exists(user_session):
        os.makedirs(user_session)
    filename = f'{user_session}/content.txt'
    with open(filename, "w") as file:
        file.write(full_text)
    
    return all_chunks

docss = get_text_chunks(docs, 'surya')

def elastic_store(docs, user_session):
    db = ElasticsearchStore.from_documents(
    docs,
    es_cloud_id="1964ec43953b4c07957e65d979ba0958:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGM1ZTk5M2RjYzIzZTQ5Mzg4NGQzYzY3Zjk5NzczODY5JGY4MjQ4NTI3MDE4YzQ5NmI4MTcwMmEzMGZiM2E3MmUz",
        index_name=user_session,
            es_api_key="U0VtaTlKQUJFM3llVzNOT1VrZ2M6T1ZteHR4WUZSRWV0RkxyVFJ5TXNFQQ=="
                )
    db.client.indices.refresh(index=user_session)

def get_vector_store(text_chunks, usersession):
    try:
        logging.info('creating vector store')
        #embeddings = OllamaEmbeddings(model="mxbai-embed-large")
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        logging.info('embedding model chosen')
        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
        logging.info(f'faiss vector created')
        vector_store.save_local(usersession)
        logging.info(f'FAISS index stored to local: {usersession}')
    except Exception as e:
        logging.info(e)
        raise

def store_vector(raw_text, user_session):
    text_chunks = get_text_chunks(raw_text, user_session)
    logging.info('text converted to chunks')
    # Store to FAISS index
    get_vector_store(text_chunks, user_session)

    # Store Elastic Search index
    if elastic_search_client.indices.exists(index=user_session):
        elastic_search_client.indices.delete(index=user_session)
        logging.info(f"Index '{user_session}' deleted.")
    elastic_search_client.indices.create(index=user_session)
    logging.info(f"Index '{user_session}' created successfully.")
    elastic_store(text_chunks, user_session)
    logging.info("Chunks stored to Elastic Search")

 

store_vector(docss, 'surya')

I0000 00:00:1722939386.148771   42910 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [6]:
len(docss)

37

In [7]:
print(max([len(chunk.page_content) for chunk in docss]))

1998


Let's convert our `Chroma` vectorstore into a retriever with the `.as_retriever()` method.

In [8]:
from langchain_elasticsearch.retrievers import ElasticsearchRetriever
from langchain.retrievers import EnsembleRetriever
from elasticsearch import Elasticsearch
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama 
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
import json
import requests
import logging
import time
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

In [9]:
def dynamic_body_func(query):
    return {
        "size": 1,
        "query": {
            "match": {
                "content": query
            }
        },
        "_source": {
            "includes": ["content"]
        }
    }

In [10]:
es_weight = 0.6
faiss_weight = 0.4
weights = [es_weight,faiss_weight]
llm = ChatOllama(model="llama3.1")
client=Elasticsearch(cloud_id="1964ec43953b4c07957e65d979ba0958:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGM1ZTk5M2RjYzIzZTQ5Mzg4NGQzYzY3Zjk5NzczODY5JGY4MjQ4NTI3MDE4YzQ5NmI4MTcwMmEzMGZiM2E3MmUz",api_key="U0VtaTlKQUJFM3llVzNOT1VrZ2M6T1ZteHR4WUZSRWV0RkxyVFJ5TXNFQQ==")
esret = ElasticsearchRetriever(es_client=client,body_func=dynamic_body_func,  es_cloud_id="1964ec43953b4c07957e65d979ba0958:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGM1ZTk5M2RjYzIzZTQ5Mzg4NGQzYzY3Zjk5NzczODY5JGY4MjQ4NTI3MDE4YzQ5NmI4MTcwMmEzMGZiM2E3MmUz", index_name='surya', es_api_key="U0VtaTlKQUJFM3llVzNOT1VrZ2M6T1ZteHR4WUZSRWV0RkxyVFJ5TXNFQQ==", content_field="content")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
new_db = FAISS.load_local('surya', embeddings, allow_dangerous_deserialization=True).as_retriever()
base_retriever = ensemble_retriever = EnsembleRetriever(retrievers=[esret, new_db], weights=weights)


Now to give it a test!

In [11]:
relevant_docs = base_retriever.get_relevant_documents('tell me about Westertoren clock')

/home/asura/miniconda3/envs/qdoc/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [12]:
len(relevant_docs)

4

## Creating a Retrieval Augmented Generation Prompt

Now we can set up a prompt template that will be used to provide the LLM with the necessary contexts, user query, and instructions!

In [13]:
from langchain.prompts import ChatPromptTemplate

template = prompt_template = """
Answer the question as detailed as possible but only on the provided context.  Review the chat history carefully to provide all necessary details and avoid incorrect information. Treat synonyms or similar words as equivalent within the context. For example, if a question refers to "modules" or "units" instead of "chapters" or "doc" instead of "document" consider them the same. 
If the question is not related to the provided context, simply respond that the question is out of context and instead provide a summary of the document and example questions that the user can ask.
Do not make up an answer if the provided question is not within the context. Instead, provide example questions that the user can ask and summary of the document.
Do not repeat facts in the answer if you have already stated them. 
If the question is short, like it is asking for the dates, names or requires a very short answer then keep the response short and to the point. 
If the question asks for a particular keyword that is in context, state information related to that keyword. 
Example: Question : When was bill gates born?
Answer: According to the documents you have uploaded, bill gates was born in 1989. 
However, if the question requires a detailed answer, then consider all possibiliies related to the question and try to answer in Bullet points and clear and concise paragraphs. 
If asked to summarise the document, try to provide a basic summary of the entire context and cover it in bullet points but keep the answer concise and not too long. 
If the question mention 'what are the contents of the file' or asks about the uploaded document or anything similar, just cover the entire document as a summary.
VERY IMPORTANT State the source in the end along with the answer but dont state the source if the question is out of context.
If the source is like : temp/abc.docx, then just mention the file name like abc.docx.
In the beginning of the anser, always mention the exact line in quotation marks that is being referred to in the answer and enclose it within **bold** tags.
Example : "According to the document in the line "The company was started in 1990", to answer your question(state the query in a shorter and concise manner), the company was founded in 1990."
Highlight Key Points: Enclose each identified key point within `**bold**` tags.
Highlight Keywords: Enclose each identified keyword within `*italic*` tags.
Documents:\n{context}\n
Question:\n{question}\n
Answer:
"""


prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll follow *exactly* the chain we made on Tuesday to keep things simple for now - if you need a refresher on what it looked like - check out last week's notebook!

In [14]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

model = ChatOllama(temperature=0.2, model="llama3.1", top_p=0.5, top_k=10)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | model, "context": itemgetter("context")}
)

Let's test it out!

In [15]:
question = "tell me about Westertoren clock."

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result)

{'response': AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-06T10:18:50.846549926Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 125940596792, 'load_duration': 1288213482, 'prompt_eval_count': 1026, 'prompt_eval_duration': 124560063000, 'eval_count': 1, 'eval_duration': 27000}, id='run-86fc5c29-5419-48d2-a150-fec25dd54a05-0'), 'context': [Document(metadata={'source': 'diary.pdf', 'page': 4, 'header': ''}, page_content='hadn’t eaten a hot meal all day, but we didn’t care; Mother and Margot were too\ntired and keyed up to eat, and Father and I were too busy.\nTuesday morning we started where we left off the night before. Bep and Miep\nwentgrocery shopping with our ration coupons, Father worked on our blackout\nscreens, we scrubbed the kitchen floor, and were once again busy from sunup to\nsundown. Until Wednesday, I didn’t have a chance to think about the enormous\nchange in my life.\nThen f

### Ground Truth Dataset Creation Using GPT-3.5-turbo and GPT-4

The next section might take you a long time to run, so the evaluation dataset is provided.

The basic idea is that we can use LangChain to create questions based on our contexts, and then answer those questions.

Let's look at how that works in the code!

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [ ]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [ ]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [ ]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
What is the aim of the paper 'A Survey on Retrieval-Augmented Text Generation'?


In [ ]:
!pip install -q -U tqdm

In [ ]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(docs[:10]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 10/10 [00:35<00:00,  3.55s/it]


In [ ]:
qac_triples[5]

{'question': 'What is the main focus of this paper?',
 'context': Document(page_content='thisisjcykcd@gmail.com, brandenwang@tencent.com\nlemaoliu@gmail.com\nAbstract\nRecently, retrieval-augmented text generation\nattracted increasing attention of the compu-\ntational linguistics community.\nCompared', metadata={'Published': '2022-02-13', 'Title': 'A Survey on Retrieval-Augmented Text Generation', 'Authors': 'Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu', 'Summary': 'Recently, retrieval-augmented text generation attracted increasing attention\nof the computational linguistics community. Compared with conventional\ngeneration models, retrieval-augmented text generation has remarkable\nadvantages and particularly has achieved state-of-the-art performance in many\nNLP tasks. This paper aims to conduct a survey about retrieval-augmented text\ngeneration. It firstly highlights the generic paradigm of retrieval-augmented\ngeneration, and then it reviews notable approaches according 

In [ ]:
answer_generation_llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
The focus of this paper is on retrieval-augmented text generation, which has gained increasing attention in the computational linguistics community. The paper conducts a survey of this field, highlighting the generic paradigm of retrieval-augmented generation, reviewing notable approaches across various tasks such as dialogue response generation and machine translation, and discussing future research directions.
question
What is the focus of the paper titled 'A Survey on Retrieval-Augmented Text Generation'?


In [ ]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 10/10 [01:10<00:00,  7.09s/it]


In [ ]:
!pip install -q -U datasets

In [ ]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 10
})

In [ ]:
eval_dataset[0]

{'question': 'What is the focus of this paper?',
 'context': 'A Survey on Retrieval-Augmented Text Generation\nHuayang Li♥,∗\nYixuan Su♠,∗\nDeng Cai♦,∗\nYan Wang♣,∗\nLemao Liu♣,∗\n♥Nara Institute of Science and Technology\n♠University of Cambridge\n♦The Chinese University of Hong Kong\n♣Tencent AI Lab',
 'ground_truth': 'The focus of this paper is on retrieval-augmented text generation, which has gained increasing attention in the computational linguistics community. The paper surveys the paradigm of retrieval-augmented generation, reviews notable approaches across various NLP tasks such as dialogue response generation and machine translation, and discusses future research directions in this area.'}

In [ ]:
eval_dataset.to_csv("groundtruth_eval_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

7359

### Evaluating RAG Pipelines

If you skipped ahead and need to load the `.csv` directly - uncomment the code below.

If you're using Colab to do this notebook - please ensure you add it to your session files.

In [16]:
from datasets import Dataset
eval_dataset = Dataset.from_csv("groundtruth_eval_dataset.csv")

In [17]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 10
})

### Evaluation Using RAGAS

Now we can evaluate using RAGAS!

The set-up is fairly straightforward - we simply need to create a dataset with our generated answers and our contexts, and then evaluate using the framework.

In [18]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate

def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

Lets create our dataset first:

In [19]:
from tqdm import tqdm
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)

100%|██████████| 10/10 [23:29<00:00, 140.94s/it]


In [20]:
basic_qa_ragas_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 10
})

In [21]:
basic_qa_ragas_dataset[0]

{'question': 'What items were packed by Miep for Anne and her family, and what was the significance of these items in the context of their situation?',
 'answer': ' **"Miep arrived and promised to return later that night, taking with her a bag full of shoes, dresses, jackets, underwear and stockings."**\n\nAccording to the diary entry, Miep packed a bag for Anne and her family containing various essential items such as:\n\n* Shoes\n* Dresses\n* Jackets\n* Underwear\n* Stockings\n\nThese items were significant in the context of their situation because they represented a way for the family to take more clothes with them without arousing suspicion. As Anne mentions, "No Jew in our situation would dare leave the house with a suitcase full of clothes." By packing these essential items, Miep helped the family to prepare for their hiding place while maintaining a sense of normalcy and avoiding drawing attention to themselves.\n\nThe packed bag also served as a way to maintain some semblance o

Save it for later:

In [22]:
basic_qa_ragas_dataset.to_csv("uniquery_ragas_dataset.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 115.31ba/s]


74259

And finally - evaluate how it did!

In [23]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

Evaluating: 100%|██████████| 60/60 [00:44<00:00,  1.35it/s]


In [24]:
basic_qa_result

{'context_precision': 1.0000, 'faithfulness': 0.3000, 'answer_relevancy': 0.3496, 'context_recall': 0.9333, 'answer_correctness': 0.2736, 'answer_similarity': 0.7441}

### Testing Other Retrievers

Now we can test our how changing our Retriever impacts our RAGAS evaluation!

We'll build this simple qa_chain factory to create standardized qa_chains where the only different component will be the retriever.

In [ ]:
def create_qa_chain(retriever):
  primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
  created_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )

  return created_qa_chain

#### Parent Document Retriever

One of the easier ways we can imagine improving a retriever is to embed our documents into small chunks, and then retrieve a significant amount of additional context that "surrounds" the found context.

You can read more about this method [here](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever)!

The basic outline of this retrieval method is as follows:

1. Obtain User Question
2. Retrieve child documents using Dense Vector Retrieval
3. Merge the child documents based on their parents. If they have the same parents - they become merged.
4. Replace the child documents with their respective parent documents from an in-memory-store.
5. Use the parent documents to augment generation.

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vectorstore = Chroma(collection_name="split_parents", embedding_function=OpenAIEmbeddings())

store = InMemoryStore()

/home/asura/miniconda3/envs/qdoc/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
parent_document_retriever.add_documents(base_docs)

Let's create, test, and then evaluate our new chain!

In [ ]:
parent_document_retriever_qa_chain = create_qa_chain(parent_document_retriever)

In [ ]:
parent_document_retriever_qa_chain.invoke({"question" : "What is RAG?"})["response"].content

'RAG stands for Retrieval-Augmented Generation.'

In [ ]:
pdr_qa_ragas_dataset = create_ragas_dataset(parent_document_retriever_qa_chain, eval_dataset)

100%|██████████| 10/10 [00:17<00:00,  1.80s/it]


In [ ]:
pdr_qa_ragas_dataset.to_csv("pdr_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

55620

In [ ]:
pdr_qa_result = evaluate_ragas_dataset(pdr_qa_ragas_dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [01:01<00:00, 61.80s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [01:09<00:00, 69.23s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [00:04<00:00,  4.02s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:06<00:00,  6.83s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.54it/s]


In [ ]:
pdr_qa_result

{'context_precision': 0.6972, 'faithfulness': 0.3500, 'answer_relevancy': 0.9439, 'context_recall': 1.0000, 'context_relevancy': 0.0134, 'answer_correctness': 0.6000, 'answer_similarity': 1.0000}

#### Ensemble Retrieval

Next let's look at ensemble retrieval!

You can read more about this [here](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)!

The basic idea is as follows:

1. Obtain User Question
2. Hit the Retriever Pair
    - Retrieve Documents with BM25 Sparse Vector Retrieval
    - Retrieve Documents with Dense Vector Retrieval Method
3. Collect and "fuse" the retrieved docs based on their weighting using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm into a single ranked list.
4. Use those documents to augment our generation.

Ensure your `weights` list - the relative weighting of each retriever - sums to 1!

In [ ]:
!pip install -q -U rank_bm25

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=75)
docs = text_splitter.split_documents(base_docs)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embedding)
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.75, 0.25])

In [ ]:
ensemble_retriever_qa_chain = create_qa_chain(ensemble_retriever)

In [ ]:
ensemble_retriever_qa_chain.invoke({"question" : "What is RAG?"})["response"].content

'RAG stands for Retrieval-Augmented Generation.'

In [ ]:
ensemble_qa_ragas_dataset = create_ragas_dataset(ensemble_retriever_qa_chain, eval_dataset)

100%|██████████| 10/10 [00:20<00:00,  2.07s/it]


In [ ]:
ensemble_qa_ragas_dataset.to_csv("ensemble_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

22820

In [ ]:
ensemble_qa_result = evaluate_ragas_dataset(ensemble_qa_ragas_dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [01:01<00:00, 61.76s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [01:08<00:00, 68.62s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [01:02<00:00, 62.45s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:08<00:00,  9.00s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.57it/s]


In [ ]:
ensemble_qa_result

{'context_precision': 0.8858, 'faithfulness': 0.7000, 'answer_relevancy': 0.8918, 'context_recall': 0.9800, 'context_relevancy': 0.0192, 'answer_correctness': 0.7750, 'answer_similarity': 1.0000}

### Conclusion

Observe your results in a table!

In [ ]:
basic_qa_result

{'context_precision': 0.5000, 'faithfulness': 0.4000, 'answer_relevancy': 0.9535, 'context_recall': 1.0000, 'context_relevancy': 0.0559, 'answer_correctness': 0.6167, 'answer_similarity': 1.0000}

In [ ]:
pdr_qa_result

{'context_precision': 0.6972, 'faithfulness': 0.3500, 'answer_relevancy': 0.9439, 'context_recall': 1.0000, 'context_relevancy': 0.0134, 'answer_correctness': 0.6000, 'answer_similarity': 1.0000}

In [ ]:
ensemble_qa_result

{'context_precision': 0.8858, 'faithfulness': 0.7000, 'answer_relevancy': 0.8918, 'context_recall': 0.9800, 'context_relevancy': 0.0192, 'answer_correctness': 0.7750, 'answer_similarity': 1.0000}

We can also zoom in on each result and find specific information about each of the questions and answers.

In [ ]:
ensemble_qa_result_df = ensemble_qa_result.to_pandas()

In [ ]:
ensemble_qa_result_df

,question,contexts,answer,ground_truths,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy,answer_correctness,answer_similarity
0,What is the focus of this paper?,[has to make an important career decision.\nNe...,The focus of this paper is on a framework call...,[The focus of this paper is on retrieval-augme...,1.000000,0.666667,0.784617,1.0,0.000000,0.50,True
1,What is the title of the paper?,[of War. The game was released worldwide in\nG...,"Title: Self-RAG: Learning to Retrieve, Generat...",[The title of the paper is 'A Survey on Retrie...,0.500000,1.000000,0.976911,1.0,0.000000,0.50,True
2,What is the aim of this paper?,[has to make an important career decision.\nNe...,The aim of this paper is to introduce a new fr...,[The aim of this paper is to conduct a compreh...,1.000000,0.333333,0.800732,1.0,0.078947,0.75,True
3,What is the main focus of the paper 'A Survey ...,[A Survey on Retrieval-Augmented Text Generati...,The main focus of the paper 'A Survey on Retri...,[The main focus of the paper 'A Survey on Retr...,0.679167,1.000000,0.982435,0.8,0.017857,1.00,True
4,What is the main focus of this paper?,[example of completions of the prompt by diffe...,I don't know.,[The main focus of this paper is to conduct a ...,1.000000,0.000000,0.742601,1.0,0.000000,0.50,True
5,What is the main focus of this paper?,[example of completions of the prompt by diffe...,I don't know.,[The main focus of this paper is to conduct a ...,1.000000,0.000000,0.742601,1.0,0.000000,0.50,True
6,What are the advantages of retrieval-augmented...,[attracted increasing attention of the compu-\...,The advantages of retrieval-augmented text gen...,[The advantages of retrieval-augmented text ge...,1.000000,1.000000,0.968712,1.0,0.025641,1.00,True
7,What is the main focus of the paper 'A Survey ...,[A Survey on Retrieval-Augmented Text Generati...,The main focus of the paper 'A Survey on Retri...,[The main focus of the paper 'A Survey on Retr...,0.679167,1.000000,0.982422,1.0,0.017857,1.00,True
8,What are the advantages of retrieval-augmented...,[attracted increasing attention of the compu-\...,The advantages of retrieval-augmented text gen...,[The advantages of retrieval-augmented text ge...,1.000000,1.000000,0.968731,1.0,0.025641,1.00,True
9,What are the advantages of retrieval-augmented...,[attracted increasing attention of the compu-\...,The advantages of retrieval-augmented text gen...,[The advantages of retrieval-augmented text ge...,1.000000,1.000000,0.968692,1.0,0.025641,1.00,True


We'll also look at combining the results and looking at them in a single table so we can make inferences about them!

In [ ]:
def create_df_dict(pipeline_name, pipeline_items):
  df_dict = {"name" : pipeline_name}
  for name, score in pipeline_items:
    df_dict[name] = score
  return df_dict

In [ ]:
basic_rag_df_dict = create_df_dict("basic_rag", basic_qa_result.items())

In [ ]:
pdr_rag_df_dict = create_df_dict("pdr_rag", pdr_qa_result.items())

In [ ]:
ensemble_rag_df_dict = create_df_dict("ensemble_rag", ensemble_qa_result.items())

In [ ]:
results_df = pd.DataFrame([basic_rag_df_dict, pdr_rag_df_dict, ensemble_rag_df_dict])

In [ ]:
results_df.sort_values("answer_correctness", ascending=False)

,name,context_precision,faithfulness,answer_relevancy,context_recall,context_relevancy,answer_correctness,answer_similarity
2,ensemble_rag,0.885833,0.70,0.891845,0.98,0.019158,0.775000,1.0
0,basic_rag,0.500000,0.40,0.953475,1.00,0.055904,0.616667,1.0
1,pdr_rag,0.697222,0.35,0.943909,1.00,0.013386,0.600000,1.0


### ❓QUESTION❓

What conclusions can you draw about the above results?

Describe in your own words what the metrics are expressing.

In [ ]:
retrieval_augmented_qa_chain = (
    RunnableParallel({
        'context': itemgetter('question') | base_retriever,
        'question': RunnablePassthrough()
    }) | {
        'response': prompt | primary_qa_llm | parser,
        'context': itemgetter('context')
    }
)